In [12]:
import pandas as pd
from Predictor import Predictor
import MDP
from utils import REGION_LABEL, REGION_CODE_INVERSE, POLICY_MAP

## Load Data

In [2]:
data = pd.read_csv('./data/region_monthly_electricity_consumption.csv').rename(columns={"Unnamed: 0": "region"})

In [3]:
data.head()

,region,2005-01-01,2005-02-01,2005-03-01,2005-04-01,2005-05-01,2005-06-01,2005-07-01,2005-08-01,2005-09-01,...,2023-09-01,2023-10-01,2023-11-01,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01
0,Overall,418.1,408.3,446.9,496.2,496.8,519.7,497.7,489.3,477.7,...,449.7,463,457.3,415.9,384.1,394.9,423.7,491,482.9,464.1
1,Central Region,485.4,459,500.2,566.4,573.3,590.1,568.3,551.7,539.5,...,505.4,523.1,527.6,481.4,442.7,436.7,478.9,553.5,558.8,524.8
2,Bishan,433.6,420.5,479.2,513.5,516.3,545,522.6,507.7,505.8,...,463,495.7,471.5,429.1,379.6,416.1,471.4,498,485.4,489.9
3,Bukit Merah,320,264.9,292.4,358.7,354.2,350,358.4,337.8,343.2,...,368.9,364.2,384.9,346.4,341.6,301.1,330.2,389.9,421.9,380.2
4,Bukit Timah,963.4,930.5,1035.2,1149.5,1132.3,1173.7,1101.2,1051.2,1039,...,911.5,951,916.9,834.9,797.2,760.7,876.8,1012.1,938.6,856.5


## Train Time Series Predictor

In [4]:
predictor = Predictor(data)

22:55:22 - cmdstanpy - INFO - Chain [1] start processing
22:55:22 - cmdstanpy - INFO - Chain [1] done processing
22:55:22 - cmdstanpy - INFO - Chain [1] start processing
22:55:22 - cmdstanpy - INFO - Chain [1] done processing
22:55:22 - cmdstanpy - INFO - Chain [1] start processing
22:55:22 - cmdstanpy - INFO - Chain [1] done processing
22:55:22 - cmdstanpy - INFO - Chain [1] start processing
22:55:22 - cmdstanpy - INFO - Chain [1] done processing
22:55:22 - cmdstanpy - INFO - Chain [1] start processing
22:55:22 - cmdstanpy - INFO - Chain [1] done processing
22:55:22 - cmdstanpy - INFO - Chain [1] start processing
22:55:22 - cmdstanpy - INFO - Chain [1] done processing
22:55:22 - cmdstanpy - INFO - Chain [1] start processing
22:55:22 - cmdstanpy - INFO - Chain [1] done processing
22:55:22 - cmdstanpy - INFO - Chain [1] start processing
22:55:22 - cmdstanpy - INFO - Chain [1] done processing
22:55:22 - cmdstanpy - INFO - Chain [1] start processing
22:55:22 - cmdstanpy - INFO - Chain [1]

In [5]:
predictor.predcit("Geylang", "2025-06")

Geylang 2025-06 -> 452.65566823496516 [434.96666666666664, 475.2]


1

## Run MDP

In [6]:
P = MDP.make_transition_prob(data)

In [7]:
R = MDP.make_rewards()

In [8]:
mdp = MDP.MDP(P, R, 0.9)

In [9]:
policy = mdp.get_policy()

In [10]:
policy

(0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2)

## Optimize

In [13]:
for region in REGION_LABEL:
    code = predictor.predcit(region, "2024-12")
    region_type = REGION_CODE_INVERSE[code // 4]
    action = POLICY_MAP[policy[code]]
    print(f"{region} - {region_type}: {action}")

Central Region 2024-12 -> 478.18334078363034 [504.26666666666665, 551.5]
Central Region - residential: Increase supply
Bishan 2024-12 -> 433.57265202351266 [457.1333333333333, 498.3666666666666]
Bishan - residential: Keep supply as this
Bukit Merah 2024-12 -> 339.40982048672595 [339.23333333333335, 367.59999999999997]
Bukit Merah - residential: Keep supply as this
Bukit Timah 2024-12 -> 846.0826086477181 [916.5333333333333, 999.8999999999999]
Bukit Timah - industrial: Decrease supply
Downtown Core 2024-12 -> 497.23452010762946 [423.96666666666664, 482.73333333333323]
Downtown Core - commercial: Increase supply
Geylang 2024-12 -> 396.52587189029015 [434.96666666666664, 475.2]
Geylang - residential: Keep supply as this
Kallang 2024-12 -> 372.36224317878924 [392.6333333333333, 428.5]
Kallang - residential: Increase supply
Marine Parade 2024-12 -> 552.0446087084254 [574.4666666666667, 620.2666666666667]
Marine Parade - residential: Keep supply as this
Museum 2024-12 -> 500.31894940550677 [